### xESMF - problems when creating a `xarray.Dataset` when using `xesmf.Regridder.__call__`

* `xesmf.Regridder.__call__` does not forward some coordinate variables to the regridded `xarray.Dataset` that are registered as `xarray.Dataset.data_vars` and not as `xarray.Dataset.coords` while it forwards others (like horizontal bounds) that should rather be replaced.
* Trying to use `xarray.open_dataset(path_to_ds, decode_coords='all')` registers all coordinate variables properly BUT removes significant metadata, not allowing `cf_xarray.accessor._get_item.drop_bounds` to distinguish between the coordinate variable and its bounds. Also it registers for example `ps` (surface pressure), that is required for a sigma hybrid vertical axis, under `xarray.Dataset.coords`, preventing `xESMF` from remapping it (it gets dropped).
* I am not aware of a `cf_xarray.CFaccessor.method` / `cf_xarray`-function that redefines / resets `xarray.Dataset.data_vars` and `xarray.Dataset.coords` of an `xarray.Dataset` in the desired manner. So I set up a custom function making use of `cf_xarray`.

In [1]:
import numpy as np
import xarray as xr
import cf_xarray as cfxr
import xesmf as xe
print("Using cf-xarray in version %s" % cfxr.__version__)
print("Using xESMF in version %s" % xe.__version__)
#print(xe.__file__)

import warnings
warnings.simplefilter("ignore") 
#with warnings.catch_warnings():
#        warnings.simplefilter("ignore")

xr.set_options(display_style='html');

Using cf-xarray in version 0.6.2.dev1+g93041e1
Using xESMF in version 0.6.1


## 1 Default approach

### Load the dataset

In [2]:
ds_path_o3 = "o3_AERmon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc"
ds_path_tos = "tos_Omon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc"
ds_o3 = xr.open_dataset(ds_path_o3).isel(time=0)
ds_tos = xr.open_dataset(ds_path_tos).isel(time=0)

In [3]:
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 47, lon: 192)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (bnds) datetime64[ns] ...
    lev_bnds   (lev, bnds) float64 ...
    ap         (lev) float64 ...
    b          (lev) float64 ...
    ps         (lat, lon) float32 ...
    ap_bnds    (lev, bnds) float64 ...
    b_bnds     (lev, bnds) float64 ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    o3         (lev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076

In [4]:
ds_tos

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 256, j: 220, vertices: 4)
Coordinates:
    time                datetime64[ns] 2010-01-16T12:00:00
  * j                   (j) int32 0 1 2 3 4 5 6 ... 213 214 215 216 217 218 219
  * i                   (i) int32 0 1 2 3 4 5 6 ... 249 250 251 252 253 254 255
    latitude            (j, i) float64 ...
    longitude           (j, i) float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (bnds) datetime64[ns] ...
    vertices_latitude   (j, i, vertices) float64 ...
    vertices_longitude  (j, i, vertices) float64 ...
    tos                 (j, i) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/b11b872a-7ab3-4ca1-8ec7-1f6936b5f354

### Calculate the regridding weights

In [5]:
# Specify a global 1 deg grid as target grid
ds_out = xe.util.grid_global(1,1)
ds_out

<xarray.Dataset>
Dimensions:  (x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

In [6]:
# Create regridding weights
def regrid(ds_in, ds_out, method='nearest_s2d', locstream_in=False):
    """Convenience function for calculating regridding weights"""
    return xe.Regridder(ds_in, ds_out, method, locstream_in)

In [7]:
regridder_o3 = regrid(ds_o3, ds_out)
regridder_o3

xESMF Regridder 
Regridding algorithm:       nearest_s2d 
Weight filename:            nearest_s2d_96x192_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (96, 192) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

In [8]:
regridder_tos = regrid(ds_tos, ds_out)
regridder_tos

xESMF Regridder 
Regridding algorithm:       nearest_s2d 
Weight filename:            nearest_s2d_220x256_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (220, 256) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

### Perform remapping

Important vertical coordinate variables are lost in the regridded dataset `ds_o3_g1`!
While the old bounds are kept in the regridded dataset `ds_tos_g1`.

In [9]:
ds_tos_g1 = regridder_tos(ds_tos, keep_attrs=True)
ds_o3_g1 = regridder_o3(ds_o3, keep_attrs=True)

In [10]:
ds_tos_g1

<xarray.Dataset>
Dimensions:             (vertices: 4, x: 360, y: 180)
Coordinates:
    time                datetime64[ns] 2010-01-16T12:00:00
    lon                 (y, x) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
    lat                 (y, x) float64 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5
Dimensions without coordinates: vertices, x, y
Data variables:
    vertices_latitude   (vertices, y, x) float64 -83.78 -83.78 ... 89.57 89.57
    vertices_longitude  (vertices, y, x) float64 233.7 233.7 ... 235.9 235.9
    tos                 (y, x) float32 nan nan nan nan ... -1.9 -1.9 -1.9 -1.9
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/b11b872a-7ab3-4ca1-8ec7-1f6936b5f354
    regrid_method:          nearest_s2d

In [11]:
ds_o3_g1

<xarray.Dataset>
Dimensions:  (lev: 47, x: 360, y: 180)
Coordinates:
    time     datetime64[ns] 2010-01-16T12:00:00
  * lev      (lev) float64 0.9961 0.9826 0.959 ... 0.0001098 4.225e-05 9.816e-06
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: x, y
Data variables:
    ps       (y, x) float32 6.666e+04 6.673e+04 ... 1.017e+05 1.017e+05
    o3       (lev, y, x) float32 1.836e-08 1.837e-08 ... 1.207e-07 1.207e-07
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076
    regrid_method:          nearest_s2d

### What can be done?

* Make all coordinate variables and auxillary coordinate variables to be recognized as coordinates by xarray.
  * by using the option decode_coordinates when loading the dataset
  * by making use of a custom function and cf_xarray
* Store the remapped variables in ds_out and transfer all necessary (eg. non-horizontal) coordinate variables to ds_out

## 2 Approach using `xarray.open_dataset(path_to_ds, decode_coords='all')`

That causes the variable `ps` in `ds_o3` to be defined under `xarray.Dataset.coords`, leading to xESMF dropping this variable when calling `xesmf.Regridder.__call__`.
For `ds_tos` the `cf_xarray.CFAccessor` cannot uniquely identify the horizontal coordinates, since xarray drops a few essential metadata upon loading the data and decoding the coordinates. 

In [12]:
ds_o3 = xr.open_dataset(ds_path_o3, decode_coords='all').isel(time=0)
ds_tos = xr.open_dataset(ds_path_tos, decode_coords='all').isel(time=0)

In [13]:
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 47, lon: 192)
Coordinates: (12/13)
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] ...
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
    lev_bnds   (lev, bnds) float64 ...
    ap         (lev) float64 ...
    b          (lev) float64 ...
    ...         ...
    ap_bnds    (lev, bnds) float64 ...
    b_bnds     (lev, bnds) float64 ...
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 ...
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 ...
Dimensions without coordinates: bnds
Data variables:
    o3         (lev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076

In [14]:
ds_tos

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 256, j: 220, vertices: 4)
Coordinates:
    time                datetime64[ns] 2010-01-16T12:00:00
    time_bnds           (bnds) datetime64[ns] ...
  * j                   (j) int32 0 1 2 3 4 5 6 ... 213 214 215 216 217 218 219
  * i                   (i) int32 0 1 2 3 4 5 6 ... 249 250 251 252 253 254 255
    latitude            (j, i) float64 ...
    longitude           (j, i) float64 ...
    vertices_latitude   (j, i, vertices) float64 ...
    vertices_longitude  (j, i, vertices) float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    tos                 (j, i) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/b11b872a-7ab3-4ca1-8ec7-1f6936b5f354

In [15]:
regridder_o3 = regrid(ds_o3, ds_out)
regridder_o3

xESMF Regridder 
Regridding algorithm:       nearest_s2d 
Weight filename:            nearest_s2d_96x192_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (96, 192) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

In [16]:
# variable ps is dropped, but should be remapped as well
ds_o3_g1 = regridder_o3(ds_o3, keep_attrs=True)
ds_o3_g1

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 47, x: 360, y: 180)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] 2010-01-01 2010-02-01
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 ... 1.963e-05 1.963e-05 0.0
    ap         (lev) float64 0.0 0.0 36.03 171.8 ... 23.15 11.12 4.281 0.9946
    b          (lev) float64 0.9961 0.9826 0.9586 0.9259 ... 0.0 0.0 0.0 0.0
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 ... 6.572 1.989 1.989 0.0
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 ... 0.0 0.0 0.0 0.0
    lon        (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat        (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: bnds, x, y
Data variables:
    o3         (lev, y, x) float32 1.836e-08 1.837e-08 ... 1.207e-07 1.207e-07
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076
    regrid_method:          nearest_s2d

In [17]:
# cf_xarray cannot identify longitude or latitude:
ds_tos.cf["latitude"]

KeyError: "Receive multiple variables for key 'latitude': {'vertices_latitude', 'latitude'}. Expected only one. Please pass a list ['latitude'] instead to get all variables matching 'latitude'."

In [18]:
ds_tos.cf.bounds

{}

## 3 Approach using custom function to reset `xarray.Dataset.data_vars` and `xarray.Dataset.coords` (making use of `cf_xarray`)

Variable `ps` in `ds_o3` is not remapped as well, all necessary coordinate variables are kept.
For `ds_tos` the horizontal bounds are dropped.

In [19]:
# It still matters that the datasets have been loaded without using xarray to decode the coords.
#  Since that drops essential metadata for cf_xarray to do its magic.
def set_data_vars_and_coords(ds):
    "Set all non data vars as coordinates and all non-coords as data_vars."
    to_coord = []
    to_datavar = []
    gridtype=""
    
    #########################################
    # Usually dealt with in another routine within clisops.core.regrid.Grid
    # (horizontal grids without lon or lat coord would be filtered out):
    lat=cfxr.accessor._get_with_standard_name(ds, "latitude")#[0]
    lon=cfxr.accessor._get_with_standard_name(ds, "longitude")#[0]
    lat=lat[0]
    lon=lon[0]
    
    # Usually dealt with in another routine
    #  setting the gridtype, nlat, nlon, ncells
    if ds[lat].ndim == 2:
        nlat = ds[lat].shape[0]
        nlon = ds[lon].shape[1]
        ncells = nlat * nlon        
    elif ds[lat].ndim == 1:
        if (
            ds[lat].shape == ds[lon].shape
            and ds[lat].dims[0]==ds[lon].dims[0]
            and len(ds[lat].dims) == 1
        ):
            nlat = ds[lat].shape[0]
            nlon = nlat
            ncells = nlat
            gridtype = "irregular" # locstream-like arranged coordinate variables
        else:
            nlat = ds[lat].shape[0]
            nlon = ds[lon].shape[0]
            ncells = nlat * nlon            
    else:
        return
    #########################################
    
    # Check by horizontal shape
    if ds[lat].ndim == 2:
        for data_var in ds.data_vars:
            if ds[data_var].ndim < 2:
                to_coord.append(data_var)
            elif ds[data_var].shape[-2:] != ds[lat].shape:
                to_coord.append(data_var)
    elif ds[lat].ndim == 1:
        for data_var in ds.data_vars:
            if gridtype == "irregular":                
                if ( len(ds[data_var].shape)>0 
                    and (ds[data_var].shape[-1],) != ds[lat].shape
                   ):
                    to_coord.append(data_var)
            else:
                if not (
                    ds[data_var].shape[-2:] == (nlat, nlon)
                    or ds[data_var].shape[-2:] == (nlon, nlat)
                ):
                    to_coord.append(data_var)
    
    # Check by attributes and names
    bounds=[]
    for bnds in ds.cf.bounds.values(): bounds+=bnds
    for bnds in bounds:
        if bnds in ds.data_vars:
            to_coord.append(bnds)   
    
    for var in ds.coords:        
        if var not in [lat, lon] + bounds:
            if gridtype=="irregular":                
                if (len(ds[var].shape)>0 and
                    ( ds[var].shape[-1]==ncells
                     and ds[var].dims[-1] in ds[lat].dims
                     and var not in ds.dims
                    )):
                    to_datavar.append(var)
            else:               
                if (len(ds[var].shape)>0 and   
                    ( ds[var].shape[-2:] == (nlat, nlon)
                     or ds[var].shape[-2:] == (nlon, nlat)
                    ) and all([dim in ds[var].dims for dim in list(ds[lat].dims) + list(ds[lon].dims)])
                   ):
                    to_datavar.append(var)                    
   
    if to_coord:
        ds = ds.set_coords(list(set(to_coord)))
    if to_datavar:
        ds = ds.reset_coords(list(set(to_datavar)))
    return ds

In [20]:
ds_o3 = xr.open_dataset(ds_path_o3).isel(time=0)
ds_o3 = set_data_vars_and_coords(ds_o3)
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 47, lon: 192)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] ...
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
    lev_bnds   (lev, bnds) float64 ...
    ap         (lev) float64 ...
    b          (lev) float64 ...
    ap_bnds    (lev, bnds) float64 ...
    b_bnds     (lev, bnds) float64 ...
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 ...
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 ...
Dimensions without coordinates: bnds
Data variables:
    ps         (lat, lon) float32 ...
    o3         (lev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076

In [21]:
ds_tos = xr.open_dataset(ds_path_tos).isel(time=0)
ds_tos = set_data_vars_and_coords(ds_tos)
ds_tos

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 256, j: 220, vertices: 4)
Coordinates:
    time                datetime64[ns] 2010-01-16T12:00:00
    time_bnds           (bnds) datetime64[ns] ...
  * j                   (j) int32 0 1 2 3 4 5 6 ... 213 214 215 216 217 218 219
  * i                   (i) int32 0 1 2 3 4 5 6 ... 249 250 251 252 253 254 255
    latitude            (j, i) float64 76.36 76.3 76.24 ... -77.25 -77.39 -77.54
    longitude           (j, i) float64 312.7 312.3 311.9 ... 131.3 132.5 133.8
    vertices_latitude   (j, i, vertices) float64 ...
    vertices_longitude  (j, i, vertices) float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    tos                 (j, i) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/b11b872a-7ab3-4ca1-8ec7-1f6936b5f354

In [22]:
regridder_o3 = regrid(ds_o3, ds_out)
regridder_o3

xESMF Regridder 
Regridding algorithm:       nearest_s2d 
Weight filename:            nearest_s2d_96x192_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (96, 192) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

In [23]:
regridder_tos = regrid(ds_tos, ds_out)
regridder_tos

xESMF Regridder 
Regridding algorithm:       nearest_s2d 
Weight filename:            nearest_s2d_220x256_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (220, 256) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

In [24]:
ds_o3_g1 = regridder_o3(ds_o3, keep_attrs=True)
ds_o3_g1

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 47, x: 360, y: 180)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] 2010-01-01 2010-02-01
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 ... 1.963e-05 1.963e-05 0.0
    ap         (lev) float64 0.0 0.0 36.03 171.8 ... 23.15 11.12 4.281 0.9946
    b          (lev) float64 0.9961 0.9826 0.9586 0.9259 ... 0.0 0.0 0.0 0.0
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 ... 6.572 1.989 1.989 0.0
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 ... 0.0 0.0 0.0 0.0
    lon        (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat        (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: bnds, x, y
Data variables:
    ps         (y, x) float32 6.666e+04 6.673e+04 ... 1.017e+05 1.017e+05
    o3         (lev, y, x) float32 1.836e-08 1.837e-08 ... 1.207e-07 1.207e-07
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076
    regrid_method:          nearest_s2d

In [25]:
ds_tos_g1 = regridder_tos(ds_tos, keep_attrs=True)
ds_tos_g1

<xarray.Dataset>
Dimensions:    (bnds: 2, x: 360, y: 180)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] 2010-01-01 2010-02-01
    lon        (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat        (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: bnds, x, y
Data variables:
    tos        (y, x) float32 nan nan nan nan nan ... -1.9 -1.9 -1.9 -1.9 -1.9
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/b11b872a-7ab3-4ca1-8ec7-1f6936b5f354
    regrid_method:          nearest_s2d

## 4 Approach storing remapped data in `ds_out`

Eventually, `xarray.Dataset.attrs` and essential `xarray.Dataset.coords` have to be moved manually, since they are not transferred (even if `keep_attrs=True` is set and the coordinates are registered as `xarray.Dataset.coords`.

In [26]:
ds_out["tos"]=regridder_tos(ds_tos.tos)
ds_out["ps"]=regridder_o3(ds_o3.ps)
ds_out["o3"]=regridder_o3(ds_o3.o3, keep_attrs=True)
ds_out

<xarray.Dataset>
Dimensions:  (lev: 47, x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
    time     datetime64[ns] 2010-01-16T12:00:00
  * lev      (lev) float64 0.9961 0.9826 0.959 ... 0.0001098 4.225e-05 9.816e-06
    ap       (lev) float64 0.0 0.0 36.03 171.8 ... 23.15 11.12 4.281 0.9946
    b        (lev) float64 0.9961 0.9826 0.9586 0.9259 ... 0.0 0.0 0.0 0.0
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    tos      (y, x) float32 nan nan nan nan nan nan ... -1.9 -1.9 -1.9 -1.9 -1.9
    ps       (y, x) float32 6.666e+04 6.673e+04 ... 1.017e+05 1.017e+05
    o3       (lev, y, x) float32 1.836e-08 1.837e-08 ... 1.207e-07 1.207e-07

In [27]:
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 47, lon: 192)
Coordinates:
    time       datetime64[ns] 2010-01-16T12:00:00
    time_bnds  (bnds) datetime64[ns] 2010-01-01 2010-02-01
  * lev        (lev) float64 0.9961 0.9826 0.959 ... 4.225e-05 9.816e-06
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 ... 1.963e-05 1.963e-05 0.0
    ap         (lev) float64 0.0 0.0 36.03 171.8 ... 23.15 11.12 4.281 0.9946
    b          (lev) float64 0.9961 0.9826 0.9586 0.9259 ... 0.0 0.0 0.0 0.0
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 ... 6.572 1.989 1.989 0.0
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 ... 0.0 0.0 0.0 0.0
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 -89.5 -87.65 -87.65 ... 87.65 87.65 89.5
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 -0.9375 0.9375 0.9375 ... 357.2 357.2 359.1
Dimensions without coordinates: bnds
Data variables:
    ps         (lat, lon) float32 7.026e+04 7.02e+04 ... 1.012e+05 1.012e+05
    o3         (lev, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076

In [28]:
for key in ds_o3.attrs:
    ds_out.attrs[key]=ds_o3.attrs[key]

In [29]:
ds_out

<xarray.Dataset>
Dimensions:  (lev: 47, x: 360, x_b: 361, y: 180, y_b: 181)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
    time     datetime64[ns] 2010-01-16T12:00:00
  * lev      (lev) float64 0.9961 0.9826 0.959 ... 0.0001098 4.225e-05 9.816e-06
    ap       (lev) float64 0.0 0.0 36.03 171.8 ... 23.15 11.12 4.281 0.9946
    b        (lev) float64 0.9961 0.9826 0.9586 0.9259 ... 0.0 0.0 0.0 0.0
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    tos      (y, x) float32 nan nan nan nan nan nan ... -1.9 -1.9 -1.9 -1.9 -1.9
    ps       (y, x) float32 6.666e+04 6.673e+04 ... 1.017e+05 1.017e+05
    o3       (lev, y, x) float32 1.836e-08 1.837e-08 ... 1.207e-07 1.207e-07
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/54f71b3d-202f-4ecc-82d9-37c76f288076